# Tarea 5
- Ivan Morales, 10 de Octubre de 2025

Realizaremos dos experimentos, cada uno con child experiments

    - Gradient Boost
    - Random Forest

### Importar las librerias necesarias y establecer conexion con Databricks

In [4]:
import os, mlflow
from dotenv import load_dotenv
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer

load_dotenv(override=True) # Cargar las variables de entorno desde el archivo .env
EXPERIMENT_NAME = "/Users/ivan.morales@iteso.mx/nyc-taxi-experiments" 

mlflow.set_tracking_uri("databricks")
experiment = mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

Preparacion de los datos

In [5]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

df_train_ene = read_dataframe('../data/green_tripdata_2025-01.parquet')
df_train_feb = read_dataframe('../data/green_tripdata_2025-02.parquet')
df_val = read_dataframe('../data/green_tripdata_2025-03.parquet')

In [6]:
df_train_ene.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44218 entries, 0 to 46620
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               44218 non-null  int32         
 1   lpep_pickup_datetime   44218 non-null  datetime64[us]
 2   lpep_dropoff_datetime  44218 non-null  datetime64[us]
 3   store_and_fwd_flag     42295 non-null  object        
 4   RatecodeID             42295 non-null  float64       
 5   PULocationID           44218 non-null  object        
 6   DOLocationID           44218 non-null  object        
 7   passenger_count        42295 non-null  float64       
 8   trip_distance          44218 non-null  float64       
 9   fare_amount            44218 non-null  float64       
 10  extra                  44218 non-null  float64       
 11  mta_tax                44218 non-null  float64       
 12  tip_amount             44218 non-null  float64       
 13  tolls_

In [7]:
df_train = pd.concat([df_train_ene, df_train_feb])
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88436 entries, 0 to 46620
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               88436 non-null  int32         
 1   lpep_pickup_datetime   88436 non-null  datetime64[us]
 2   lpep_dropoff_datetime  88436 non-null  datetime64[us]
 3   store_and_fwd_flag     84590 non-null  object        
 4   RatecodeID             84590 non-null  float64       
 5   PULocationID           88436 non-null  object        
 6   DOLocationID           88436 non-null  object        
 7   passenger_count        84590 non-null  float64       
 8   trip_distance          88436 non-null  float64       
 9   fare_amount            88436 non-null  float64       
 10  extra                  88436 non-null  float64       
 11  mta_tax                88436 non-null  float64       
 12  tip_amount             88436 non-null  float64       
 13  tolls_

Concatemos los datasets de enero y febrero para construir una base mas robusta para validar con los datos de marzo.

Feature Engineering + One Hot Encoding, pipeline simple inicial

In [8]:
def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

X_val = preprocess(df_val, dv)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

Definir los dataset como objetos de *mlflow* para poderlos trackear

In [9]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01&02")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-03")

# Modelos

In [10]:
# XGBOOST 
import math
import optuna
import pathlib
import xgboost as xgb
from optuna.samplers import TPESampler
from mlflow.models.signature import infer_signature

In [11]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [12]:
# ------------------------------------------------------------
# Definir la función objetivo para Optuna
#    - Recibe un `trial`, que se usa para proponer hiperparámetros.
#    - Entrena un modelo con esos hiperparámetros.
#    - Calcula la métrica de validación (RMSE) y la retorna (Optuna la minimizará).
#    - Abrimos un run anidado de MLflow para registrar cada trial.
# ------------------------------------------------------------
def objective(trial: optuna.trial.Trial):
    # Hiperparámetros MUESTREADOS por Optuna en CADA trial.
    # Nota: usamos log=True para emular rangos log-uniformes (similar a loguniform).
    params = {
        "max_depth": trial.suggest_int("max_depth", 4, 100),
        "learning_rate": trial.suggest_float("learning_rate", math.exp(-3), 1.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha",   math.exp(-5), math.exp(-1), log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", math.exp(-6), math.exp(-1), log=True),
        "min_child_weight": trial.suggest_float("min_child_weight", math.exp(-1), math.exp(3), log=True),
        "objective": "reg:squarederror",  
        "seed": 42,                      
    }

    # Run anidado para dejar rastro de cada trial en MLflow
    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "xgboost")  # etiqueta informativa
        mlflow.log_params(params)                  # registra hiperparámetros del trial

        # Entrenamiento con early stopping en el conjunto de validación
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, "validation")],
            early_stopping_rounds=10,
        )

        # Predicción y métrica en validación
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)

        # Registrar la métrica principal
        mlflow.log_metric("rmse", rmse)

        # La "signature" describe la estructura esperada de entrada y salida del modelo:
        # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
        # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
        signature = infer_signature(X_val, y_pred)

        # Guardar el modelo del trial como artefacto en MLflow.
        mlflow.xgboost.log_model(
            booster,
            name="model",
            input_example=X_val[:5],
            signature=signature
        )

    # Optuna minimiza el valor retornado
    return rmse

Flujo de Busqueda

In [13]:
mlflow.xgboost.autolog(log_models=False)

# ------------------------------------------------------------
# Crear el estudio de Optuna
#    - Usamos TPE (Tree-structured Parzen Estimator) como sampler.
#    - direction="minimize" porque queremos minimizar el RMSE.
# ------------------------------------------------------------
sampler = TPESampler(seed=42)
study = optuna.create_study(direction="minimize", sampler=sampler)

# ------------------------------------------------------------
# Ejecutar la optimización (n_trials = número de intentos)
#    - Cada trial ejecuta la función objetivo con un set distinto de hiperparámetros.
#    - Abrimos un run "padre" para agrupar toda la búsqueda.
# ------------------------------------------------------------
with mlflow.start_run(run_name="XGBoost Hyperparameter Optimization (Optuna)", nested=True):
    study.optimize(objective, n_trials=10)

    # --------------------------------------------------------
    # Recuperar y registrar los mejores hiperparámetros
    # --------------------------------------------------------
    best_params = study.best_params
    # Asegurar tipos/campos fijos (por claridad y consistencia)
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"

    mlflow.log_params(best_params)

    # Etiquetas del run "padre" (metadatos del experimento)
    mlflow.set_tags({
        "project": "NYC Taxi Time Prediction Project",
        "optimizer_engine": "optuna",
        "model_family": "xgboost",
        "feature_set_version": 1,
    })

    # --------------------------------------------------------
    # 7) Entrenar un modelo FINAL con los mejores hiperparámetros
    #    (normalmente se haría sobre train+val o con CV; aquí mantenemos el patrón original)
    # --------------------------------------------------------
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, "validation")],
        early_stopping_rounds=10,
    )

    # Evaluar y registrar la métrica final en validación
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # --------------------------------------------------------
    # 8) Guardar artefactos adicionales (p. ej. el preprocesador)
    # --------------------------------------------------------
    pathlib.Path("preprocessor").mkdir(exist_ok=True)
    with open("preprocessor/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("preprocessor/preprocessor.b", artifact_path="preprocessor")

    # La "signature" describe la estructura esperada de entrada y salida del modelo:
    # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
    # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
    # Si X_val es la matriz dispersa (scipy.sparse) salida de DictVectorizer:
    feature_names = dv.get_feature_names_out()
    input_example = pd.DataFrame(X_val[:5].toarray(), columns=feature_names)

    # Para que las longitudes coincidan, usa el mismo slice en y_pred
    signature = infer_signature(input_example, y_val[:5])

    # Guardar el modelo del trial como artefacto en MLflow.
    mlflow.xgboost.log_model(
        booster,
        name="model",
        input_example=input_example,
        signature=signature
    )

[I 2025-10-28 20:31:52,111] A new study created in memory with name: no-name-7ffa3aec-134d-4296-a5ea-77545d4ce509


[0]	validation-rmse:6.50020
[1]	validation-rmse:6.44713
[2]	validation-rmse:6.44851
[3]	validation-rmse:6.44788
[4]	validation-rmse:6.44758
[5]	validation-rmse:6.44908
[6]	validation-rmse:6.44861
[7]	validation-rmse:6.44855
[8]	validation-rmse:6.44929
[9]	validation-rmse:6.44927
[10]	validation-rmse:6.44808
[11]	validation-rmse:6.44815


c:\Users\ivanm\5to Semestre\Proyecto de Ciencia de Datos\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:169: UserWarning: [20:32:02] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/10/28 20:32:14 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


c:\Users\ivanm\5to Semestre\Proyecto de Ciencia de Datos\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [20:32:14] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)
2025/10/28 20:32:15 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-10-28 20:32:18,318] Trial 0 finished with value: 6.448153607973611 and parameters: {'max_depth': 40, 'learning_rate': 0.8625543817410922, 'reg_alpha': 0.12593061066249622, 'reg_lambda': 0.049454235173237264, 'min_child_weight': 0.6866535292359801}. Best is trial 0 with value: 6.448153607973611.


🏃 View run stylish-squirrel-627 at: https://dbc-ed7122c4-c5b7.cloud.databricks.com/ml/experiments/2414141065980596/runs/5d85fbd06a79494db6c1f06671b3e02e
🧪 View experiment at: https://dbc-ed7122c4-c5b7.cloud.databricks.com/ml/experiments/2414141065980596
[0]	validation-rmse:9.39331
[1]	validation-rmse:9.06545
[2]	validation-rmse:8.77322
[3]	validation-rmse:8.50858
[4]	validation-rmse:8.26164
[5]	validation-rmse:8.04744
[6]	validation-rmse:7.85184
[7]	validation-rmse:7.67441
[8]	validation-rmse:7.51775
[9]	validation-rmse:7.37410
[10]	validation-rmse:7.24928
[11]	validation-rmse:7.13915
[12]	validation-rmse:7.04022
[13]	validation-rmse:6.95339
[14]	validation-rmse:6.87086
[15]	validation-rmse:6.80144
[16]	validation-rmse:6.74283
[17]	validation-rmse:6.68987
[18]	validation-rmse:6.64133
[19]	validation-rmse:6.59762
[20]	validation-rmse:6.56128
[21]	validation-rmse:6.52847
[22]	validation-rmse:6.49955
[23]	validation-rmse:6.47610
[24]	validation-rmse:6.45523
[25]	validation-rmse:6.43780
[2

...

c:\Users\ivanm\5to Semestre\Proyecto de Ciencia de Datos\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [20:35:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)


🏃 View run enchanting-shad-960 at: https://dbc-ed7122c4-c5b7.cloud.databricks.com/ml/experiments/2414141065980596/runs/5a8d8c52039340dd921ca1632d565da4
🧪 View experiment at: https://dbc-ed7122c4-c5b7.cloud.databricks.com/ml/experiments/2414141065980596


[W 2025-10-28 20:36:00,435] Trial 8 failed with parameters: {'max_depth': 15, 'learning_rate': 0.21992487468175848, 'reg_alpha': 0.007731550026907306, 'reg_lambda': 0.23377457337376373, 'min_child_weight': 1.0357439143907545} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\ivanm\5to Semestre\Proyecto de Ciencia de Datos\nyc-taxi-predictions-2025\.venv\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\ivanm\AppData\Local\Temp\ipykernel_13348\3894802457.py", line 48, in objective
    mlflow.xgboost.log_model(
    ~~~~~~~~~~~~~~~~~~~~~~~~^
        booster,
        ^^^^^^^^
    ...<2 lines>...
        signature=signature
        ^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "c:\Users\ivanm\5to Semestre\Proyecto de Ciencia de Datos\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py", line 279, in log_model
    return Model.log(
           ~~~~~~~

🏃 View run XGBoost Hyperparameter Optimization (Optuna) at: https://dbc-ed7122c4-c5b7.cloud.databricks.com/ml/experiments/2414141065980596/runs/5370f6320cd8408b96f24ea9f9076423
🧪 View experiment at: https://dbc-ed7122c4-c5b7.cloud.databricks.com/ml/experiments/2414141065980596


KeyboardInterrupt: 

In [14]:
model_name = "workspace.default.nyc-taxi-model-xgboost-tarea-5"

In [15]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name=model_name
)

Successfully registered model 'workspace.default.nyc-taxi-model-xgboost-tarea-5'.
2025/10/28 20:36:54 WARNING mlflow.tracking._model_registry.fluent: Run with id 9bdbe9a32f014581aeb6f8adb4360a9a has no artifacts at artifact path 'model', registering model based on models:/m-8c6d8c4c60374495807456c18eb16872 instead


Uploading artifacts:   0%|          | 0/8 [00:00<?, ?it/s]

Created version '1' of model 'workspace.default.nyc-taxi-model-xgboost-tarea-5'.


Asignamos alias

In [16]:
from mlflow import MlflowClient

client = MlflowClient()

In [17]:
model_version = result.version
new_alias = "Challenger"

client.set_registered_model_alias(
    name=model_name,
    alias=new_alias,
    version=result.version
)

In [18]:
from datetime import datetime

date = datetime.today()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=[], creation_timestamp=1761705419748, current_stage=None, deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description=('The model version 1 was transitioned to Challenger on 2025-10-28 '
 '20:43:30.198948'), last_updated_timestamp=1761705811659, metrics=[<Metric: dataset_digest='', dataset_name='', key='best_iteration', model_id='m-8c6d8c4c60374495807456c18eb16872', run_id='9bdbe9a32f014581aeb6f8adb4360a9a', step=0, timestamp=1761705140751, value=48.0>,
 <Metric: dataset_digest='', dataset_name='', key='rmse', model_id='m-8c6d8c4c60374495807456c18eb16872', run_id='9bdbe9a32f014581aeb6f8adb4360a9a', step=0, timestamp=1761705143719, value=6.342751049216642>,
 <Metric: dataset_digest='', dataset_name='', key='stopped_iteration', model_id='m-8c6d8c4c60374495807456c18eb16872', run_id='9bdbe9a32f014581aeb6f8adb436